In [33]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Fullscreen
#import seaborn
import holoviews as hv
import hvplot.pandas 
import panel as pn
import matplotlib as plt
%matplotlib inline
pn.extension('tabulator')
from panel.interact import interact
import param
pn.extension(sizing_mode="stretch_width", server=True)


In [34]:
#gdp = pd.read_csv("C:/Users/charl/OneDrive/Desktop/programming/panel_dashboard/species/data/gdp.csv",encoding="utf-8",delimiter=",",decimal=".")
cities = gpd.read_file(gpd.datasets.get_path("naturalearth_cities"))
food = pd.read_csv("data_stats/global-food.csv",encoding="utf-8",delimiter=",",decimal=".")
continents=gpd.read_file("C:data_stats/World_Continents.shp")

In [35]:
continents_food = ['Europe', 'North America', 'South America', 'Asia', 'Africa', 'Australia'] #group by continent


In [36]:
food_continents = food[food['Country'].isin(continents_food)]
food_continents.fillna(0)

,Product,Country,Year,Population,Production (t),production__tonnes__per_capita,Production per capita (kg),Yield (t/ha),Yield (kg/animal),Land Use (ha),...,other_uses__tonnes__per_capita,Other uses per capita (kg),Supply chain waste (t),waste_in_supply_chain__tonnes__per_capita,Supply chain waste per capita (kg),Food supply (kg per capita per year),Food supply (g per capita per day),Food supply (kcal per capita per day),Food supply (Protein g per capita per day),Food supply (Fat g per capita per day)
61,0.0,Africa,1961,291178000.0,16140043.0,0.058953,58.953470,1.045041,0.0,15444418.0,...,0.000204,0.203798,1083000.0,0.004326,4.325540,37.834770,103.656903,325.42468,8.432948,3.438252
62,0.0,Africa,1962,298334460.0,17556072.0,0.062580,62.580325,1.112496,0.0,15780790.0,...,0.000189,0.189142,1186000.0,0.004624,4.623719,37.917000,103.882191,325.15848,8.433020,3.431601
63,0.0,Africa,1963,305755600.0,17625504.0,0.061297,61.296865,1.110046,0.0,15878168.0,...,0.000232,0.231787,1133000.0,0.004310,4.310167,38.387398,105.170952,329.65600,8.542378,3.475431
64,0.0,Africa,1964,313466080.0,16145294.0,0.054766,54.765950,0.979997,0.0,16474843.0,...,0.000231,0.230642,1124000.0,0.004172,4.171520,39.256195,107.551218,337.37152,8.744979,3.557508
65,0.0,Africa,1965,321447400.0,16934360.0,0.056017,56.017000,1.027539,0.0,16480500.0,...,0.000243,0.242809,1149000.0,0.004159,4.159281,40.343780,110.530903,346.83664,8.986965,3.646669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10949,0.0,South America,2017,420982660.0,160606640.0,0.381752,381.751750,5.929682,0.0,27085204.0,...,0.009010,9.010441,11237000.0,0.026710,26.709631,28.527222,78.156772,234.93513,5.409600,1.300243
10950,0.0,South America,2018,424740740.0,139228740.0,0.328013,328.013480,5.242760,0.0,26556380.0,...,0.009124,9.124133,9563000.0,0.022530,22.529780,29.554987,80.972566,243.30025,5.626802,1.382268
10951,0.0,South America,2019,428318200.0,171818510.0,0.401417,401.416600,6.140583,0.0,27980812.0,...,0.009043,9.043003,11733000.0,0.027412,27.411604,28.619057,78.408375,233.61804,5.373773,1.278151
10952,0.0,South America,2020,431530100.0,176377540.0,0.409005,409.005250,6.034921,0.0,29226156.0,...,0.007850,7.850146,12031000.0,0.027899,27.898919,28.030333,76.795432,228.96483,5.266777,1.249702


In [37]:
food_continents_interactive=food_continents


In [38]:
# Radio buttons for  stats 
yaxis_food = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Production (t)','Yield (t/ha)','production__tonnes__per_capita'],
    button_type='success'
)

    
yaxis_food

RadioButtonGroup(button_type='success', name='Y axis', options=['Production (t)', ...], sizing_mode='stretch_width', value='Production (t)')

In [39]:
food_plot = food_continents_interactive.hvplot(x='Year', by='Country', y=yaxis_food, line_width=2, title="Food Production")

food_plot

ParamFunction(function, _pane=HoloViews, sizing_mode='stretch_width')

In [40]:
#bar plot 
food_filtered = food_continents_interactive[['Country', 'Year', 'Exports (t)', 'exports__tonnes__per_capita','Domestic supply (t)', 'domestic_supply__tonnes__per_capita','Imports (t)',
       'imports__tonnes__per_capita','Population' ]]

food_filtered_renamed=food_filtered.rename(columns={'Country':'Continent','Year':'Year','Exports (t)':'Exports(t)', 'exports__tonnes__per_capita':'Exports(t)_per_capita','Domestic supply (t)':'DomesticSupply(t)', 'domestic_supply__tonnes__per_capita':'DomesticSupply(t)_per_capita','Imports (t)':'Imports(t)','imports__tonnes__per_capita':'Imports(t)_per_capita' })

#food_bar_plot = food_continents_interactive.groupby('Country',type)
food_filtered_renamed.head()

,Continent,Year,Exports(t),Exports(t)_per_capita,DomesticSupply(t),DomesticSupply(t)_per_capita,Imports(t),Imports(t)_per_capita,Population
61,Africa,1961,1623000.0,0.006583,13377000.0,0.052300,387000.0,0.001513,291178000.0
62,Africa,1962,2864000.0,0.011340,13815000.0,0.052733,531000.0,0.002027,298334460.0
63,Africa,1963,3002000.0,0.011599,14143000.0,0.052689,386000.0,0.001438,305755600.0
64,Africa,1964,1676000.0,0.006318,14395000.0,0.052328,661000.0,0.002403,313466080.0
65,Africa,1965,598000.0,0.002199,15213000.0,0.053948,441000.0,0.001564,321447400.0


In [41]:
trade_stats = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Exports(t)', 'Imports(t)'], 
    button_type='success'
)

In [42]:
trade_stats

RadioButtonGroup(button_type='success', name='Y axis', options=['Exports(t)', ...], sizing_mode='stretch_width', value='Exports(t)')

In [43]:
trade_stats_grouped= food_filtered_renamed.groupby(['Continent','Year']).sum()
#trade_stats_grouped.reset_index(inplace=True)


In [44]:
trade_stats_grouped.head()

In [45]:
#bar_plot = trade_stats_grouped(kind='bar',x='Continent',y='Exports(t)', 'Imports(t)', title='Import & Export Stats by year')

In [46]:
plot_bars =trade_stats_grouped.hvplot(x='Year', by='Continent', y=trade_stats, line_width=2, title='Import & Export Stats per year and Continent')

plot_bars

In [47]:
continents.head()

In [48]:
merged_continents= continents.merge(food_continents, left_on='CONTINENT', right_on='Country')


In [49]:
merged_continents.head()

In [50]:

merged_filter=merged_continents[['CONTINENT', 'geometry','Year','Production (t)', 'Yield (t/ha)','production__tonnes__per_capita']]
#merged_years=merged_filter[merged_filter['Year'].isin(years)]
filtered_df = merged_filter[(merged_filter['Year'] >= 2000) & (merged_filter['Year'] <= 2022)]


In [51]:
filtered_df

In [52]:
filtered_df.head() # filter per year for map
filtered_year_df = filtered_df[filtered_df['Year'] == 2020]


In [53]:
year_slider = pn.widgets.IntSlider(name="Year", start=1970, end=2019)
year_slider


In [54]:
filtered_df_i =food_filtered.interactive()

In [55]:
filtered_df_i.head()

In [56]:
#year_table= filtered_df_i[filtered_df_i['Year'] > year_slider].head(5)
#year_table


In [57]:
year_table= pn.widgets.Tabulator(filtered_df_i[filtered_df_i['Year'] > year_slider],page_size=5)
year_table

In [58]:
m = filtered_year_df.explore('Yield (t/ha)', # show the Distance column
                   cmap='BuPu', # use the 'plasma' colormap from matplotlib
                   legend_kwds={'caption': 'Colormap based on Yield t(ha) for the year 2020'} # set the caption to a longer explanation
                  )
cities.explore(m=m,  color='grey')
m

In [59]:
grouped_year=food_continents_interactive.groupby(['Country','Year']).sum()

grouped_year_interactive =grouped_year

In [60]:
#scatter plot
x = pn.widgets.Select(name='Production Statistics', options=['Land Use (ha)', 'Yield (t/ha)', 'area_harvested__ha__per_capita'])
y = pn.widgets.Select(name='Food Supply Statistics', options=['Food (t)', 'food__tonnes__per_capita', 'Food supply (g per capita per day)'])

plot_scatter = grouped_year_interactive.hvplot(x=x, y=y, kind='scatter', colorbar=False, width=400,alpha=0.5,by="Country")

plot_scatter_layout = pn.Column(
    pn.pane.Markdown("### Select combinations of different variables grouped by the sum of the different years to show scatter plot", styles={'font-weight': 'bold', 'color':'grey'}),
    x,y, plot_scatter
    
)

In [61]:
plot_scatter_layout

In [62]:
@pn.cache  # gapminder data
def get_dataset():
    url = 'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
    return pd.read_csv(url)

dataset = get_dataset()

YEARS = [int(year) for year in dataset.year.unique()]

#dataset.tail(10)

In [63]:
#dashboard layout using Template

text="A simple dashboard to practice visualizations using the Panel Framework"
template = pn.template.FastListTemplate(
    title='Agricultural Production Statistics', 
    sidebar=[pn.pane.Markdown("# Dashboard showing agricultural production statistics by continent and year"),
             text,
             pn.pane.Markdown("#### Data from Our World in Data.https://ourworldindata.org"), 
             year_slider],
    main=[
        pn.Row(
            pn.Column(
            yaxis_food,
            food_plot,
            sizing_mode="stretch_width"),
            plot_scatter_layout,
            sizing_mode="stretch_width"),
    
        
        pn.Row(
            pn.Column(
            pn.pane.plot.Folium(m, height=400),
            sizing_mode="stretch_width"),
            pn.Column(   
            trade_stats,
            plot_bars,  
            sizing_mode="stretch_width"),
        ),
            
        pn.Row(
            pn.Column(
                year_table,
                sizing_mode="stretch_width"),

        )
        
    ] ,   
            header_background="#6b8e23",

)
        
        
        
template.show()
#pn.panel(template).servable(title='Agricultural Stats App')

